In [2]:
# installs selenium
!pip install selenium
from selenium.webdriver import Chrome
driver_path = 'Users/user/Downloads/chromedriver2'

In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
driver = Chrome(driver_path)
driver.get("https://careers.lse.ac.uk/students/jobs")

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


In [5]:
#navigating to the username box
username = driver.find_element("css selector", "body > div.main > div > div > div > div.sparse-body.auth-body > div > div.ch-account > div > form > div.login > div.form-group.username-group")
# clicking on the username box
username.click()
from selenium.webdriver.common.keys import Keys
#navigating to the username box
username = driver.find_element("css selector", "body > div.main > div > div > div > div.sparse-body.auth-body > div > div.ch-account > div > form > div.login > div.form-group.username-group")
# clicking on the username box
username.click()
inputElement = driver.find_element("css selector", "#username")
inputElement.send_keys('202008067')

In [6]:
password = driver.find_element("css selector", "body > div.main > div > div > div > div.sparse-body.auth-body > div > div.ch-account > div > form > div.login > div.form-group.password-group")
# clicking on the username box
password.click()
inputElement = driver.find_element("css selector", "#password")
inputElement.send_keys('Spiros135!')
inputElement.send_keys(Keys.ENTER)

In [7]:
more_search_options = driver.find_element("css selector", "#main > div.under-nav > div > div > form > div:nth-child(2) > div > div > ul > li:nth-child(4) > a > span")
more_search_options.click()

In [8]:
investment_banking = driver.find_element("css selector", "#industry > option:nth-child(29)")
investment_banking.click()
include_expired = driver.find_element("css selector", "#main > div.under-nav > div > div > form > div.form-group > div > div.col-md-12 > div.more-options.collapse.in > div:nth-child(5) > div > div:nth-child(2) > label > input[type=checkbox]")
include_expired.click()
submit_search = driver.find_element("css selector", "#main > div.under-nav > div > div > form > div:nth-child(3) > div > div > button")
submit_search.click()

In [9]:
#clicking all 6000+ jobs
show_selector = driver.find_element("css selector", " #main > div.main-content > div > div.row > div.col-md-9 > div.text-center > div > div > button")
show_selector.click()

In [10]:
url = driver.current_url
response = requests.get(url)
print(response)
url = driver.current_url
print(url)

<Response [200]>
https://careers.lse.ac.uk/students/jobs/Search?text=&typeofwork=-1&location=&country=Any&industry=1810&includeexpired=true


In [11]:
html = driver.page_source
soup = BeautifulSoup(html)
soup.find('div',attrs = {'class':'panel-heading'}).get_text()

'\n                Found 2207\n                opportunities\n\n\n                \n\n\n                                Relevance\n                            \n\n                                Most Recent\n                            \n\n                                Closing Soon\n                            \n\n\n'

In [12]:
# run this block when job_titles/links is empty to RESET 
jobs = soup.find_all('div',attrs={'class':'list-group-item'})

For-loop for data extraction

In [13]:
job_titles_1 = []
job_links_1 = []
app_date_list = []
job_loc_list = []

In [14]:
for job in jobs:
  job_title_1 = job.find('a').get_text().replace("\n                                ","").replace("\n                        ","")
  job_titles_1.append(job_title_1)

  link = 'https://careers.lse.ac.uk/' + job.find('a').get('href')
  job_links_1.append(link)    
  
  app_date = job.find('div',attrs={'class':'job-list-close'}).get_text().replace("\nClosed\n                            - ","").replace("\n                        ","").replace("\nCloses    - ","")
  app_date_list.append(app_date)

  try:
    job_loc = job.find('div',attrs={'class':'job-list-location'}).get_text().replace("\n","")
    job_loc_list.append(job_loc)

  except AttributeError:
    job_loc_list.append(" ")

In [15]:
len(job_titles_1)

10

In [16]:
len(job_links_1)

10

In [ ]:
#DATA COLLECTION AS OF 06.12.22

from pandas.core.accessor import CachedAccessor
#accessing each link and extracting our data
job_detail_list = []
job_title_list = []
company_list = []

opp_type_list = []
con_type_list = []
com_date_list = []

count = 0

for link in job_links:

  url_job = link
  response_html_job = requests.get(url_job)

  driver.get(link)
  html = driver.page_source 
  soup = BeautifulSoup(html)

  title = soup.find('h3').get_text() 
  job_title_list.append(title)    
    

# Job Details

  job_details = soup.find('div',attrs={'class':"job-details"}).get_text().replace("\n                    ","").replace("\n                ","")
  job_detail_list.append(job_details)                                                                                          

# Company Name  

  try:
    company = soup.find_all('div',attrs={'class':'container'})[2].find('a').get_text()
    company_list.append(company)

  except AttributeError:
    company = soup.find_all('div',attrs={'class':'container'})[2].find('h4').get_text().replace("\n","")
    company_list.append(company)
  
# Opportunity Type: Graduate or Undergraduate Employment

  opp_type = soup.find('div',attrs={'class':'list-group-item-heading'}).get_text().replace("\n                            ","").replace("\n                        ","").replace("\n                    ","")
  opp_type_list.append(opp_type)                                                           

# Contract Type

  try:
    con_type = soup.find_all('div',attrs={'class':'panel-body'})[3].find_all('p')[0].get_text()
    con_type_list.append(con_type)

  except IndexError: 
    try:
      con_type = soup.find_all('div',attrs={'class':'panel-body'})[2].find_all('p')[0].get_text()
      con_type_list.append(con_type) # some jobs have opportunities in the 3rd 'panel-body' block rather than the 4th

    except IndexError: 
      con_type = soup.find_all('div',attrs={'class':'panel-body'})[1].find_all('p')[0].get_text()
      con_type_list.append(con_type) # some jobs have opportunities in the 2nd 'panel-body' block rather than the 4th

# Commencement Date 

  try:
    com_date = soup.find_all('div',attrs={'class':'panel-body'})[3].find_all('p')[3].get_text()
    com_date_list.append(com_date)

  except IndexError: 
    try:
      com_date = soup.find_all('div',attrs={'class':'panel-body'})[2].find_all('p')[3].get_text()
      com_date_list.append(com_date)

    except IndexError:
      try:
        com_date = soup.find_all('div',attrs={'class':'panel-body'})[1].find_all('p')[3].get_text()
        com_date_list.append(com_date)

      except IndexError:
        try: 
          com_date = soup.find_all('div',attrs={'class':'panel-body'})[2].find_all('p')[2].get_text()
          com_date_list.append(com_date)

        except IndexError:
          try: 
            com_date = soup.find_all('div',attrs={'class':'panel-body'})[2].find_all('p')[1].get_text()
            com_date_list.append(com_date)

          except IndexError:
            com_date = soup.find_all('div',attrs={'class':'panel-body'})[1].find_all('p')[1].get_text()
            com_date_list.append(com_date)

  count = count + 1
  if count % 100 == 0:
    print(count)

Creating dataframes and saving files as .csv

In [ ]:
df_titles = pd.DataFrame(job_title_list, columns = ['title']) 
df_details = pd.DataFrame(job_detail_list, columns = ['details']) 
df_app = pd.DataFrame(app_date_list, columns = ['deadline']) 
df_opp = pd.DataFrame(opp_type_list, columns = ['opport_type']) 
df_com = pd.DataFrame(com_date_list, columns = ['commence_date']) 
df_con = pd.DataFrame(con_type_list, columns = ['contract_type'])
df_comp = pd.DataFrame(company_list, columns = ['company'])

#Concatenating the dataframes
pdList = [df_titles, df_details,df_app,df_opp,df_com,df_con,df_comp]
df_final = pd.concat(pdList,axis=1)
df_final

In [ ]:
df_final.to_csv('6622_dates.csv', encoding = 'utf-8-sig') 